In [1]:
import numpy as np 
import pandas as pd
import datetime
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import re # regex
from scipy.misc import imread
#from wordcloud import WordCloud, STOPWORDS # Wordcloud: pip install wordcloud

%matplotlib inline

In [2]:
postulantes_educacion = pd.read_csv('../data/1_postulantes_educacion_train.csv')
postulantes_genero_edad = pd.read_csv('../data/2_postulantes_genero_edad_train.csv')
vistas = pd.read_csv('../data/3_vistas_train.csv')
postulaciones = pd.read_csv('../data/4_postulaciones_train.csv')
avisos_online = pd.read_csv('../data/5_avisos_online_train.csv')
avisos_detalle = pd.read_csv('../data/6_avisos_detalle_train.csv')

# Juntamos de los postulantes la informacion sobre educacion, genero, y edad

### Obtenemos el maximo nivel de educacion para cada postulante

In [3]:
postulantes_educacion.head(2)

,idpostulante,nombre_estudio,estado_estudio
0,8BkL,Universitario,En Curso
1,1d2B,Universitario,En Curso


In [4]:
len(postulantes_educacion)

568612

In [5]:
print('Existen datos de educacion de', len(postulantes_educacion.drop_duplicates(['idpostulante'], keep='last', inplace=False)), 'postulantes')

Existen datos de educacion de 366395 postulantes


In [6]:
def cuantificar_estudios(row):
    if (row['nombre_estudio'] == 'Doctorado'):
        row['nombre_estudio'] = 7
    if (row['nombre_estudio'] == 'Master'):    
        row['nombre_estudio'] = 6
    if (row['nombre_estudio'] == 'Posgrado'):    
        row['nombre_estudio'] = 5
    if (row['nombre_estudio'] == 'Universitario'):    
        row['nombre_estudio'] = 4
    if (row['nombre_estudio'] == 'Terciario/Técnico'):    
        row['nombre_estudio'] = 3
    if (row['nombre_estudio'] == 'Secundario'):    
        row['nombre_estudio'] = 2
    if (row['nombre_estudio'] == 'Otro'):    
        row['nombre_estudio'] = 1    
        
    if (row['estado_estudio'] == 'Graduado'):
        row['estado_estudio'] = 3
    if (row['estado_estudio'] == 'En Curso'):    
        row['estado_estudio'] = 2
    if (row['estado_estudio'] == 'Abandonado'):    
        row['estado_estudio'] = 1
        
    return row    

def descuantificar_estudios(row):
    if (row['nombre_estudio'] == 7):
        row['nombre_estudio'] = 'Doctorado'
    if (row['nombre_estudio'] == 6):    
        row['nombre_estudio'] = 'Master'
    if (row['nombre_estudio'] == 5):    
        row['nombre_estudio'] = 'Posgrado'
    if (row['nombre_estudio'] == 4):    
        row['nombre_estudio'] = 'Universitario'
    if (row['nombre_estudio'] == 3):    
        row['nombre_estudio'] = 'Terciario/Técnico'
    if (row['nombre_estudio'] == 2):    
        row['nombre_estudio'] = 'Secundario'
    if (row['nombre_estudio'] == 1):    
        row['nombre_estudio'] = 'Otro'    
        
    if (row['estado_estudio'] == 3):
        row['estado_estudio'] = 'Graduado'
    if (row['estado_estudio'] == 2):    
        row['estado_estudio'] = 'En Curso'
    if (row['estado_estudio'] == 1):    
        row['estado_estudio'] = 'Abandonado'
        
    return row    

In [7]:
postulantes_educacion = postulantes_educacion.apply(lambda row: cuantificar_estudios(row), axis=1)

In [8]:
# Obtenemos max de la columna nombre
idx = postulantes_educacion.groupby(['idpostulante'])['nombre_estudio'].transform(max) == postulantes_educacion['nombre_estudio']

In [9]:
# Sobre los anteriores, obtenemos los max de la columna estado
idx_postulantes_educacion_max = postulantes_educacion[idx].groupby(['idpostulante'])['estado_estudio'].transform(max) == postulantes_educacion[idx]['estado_estudio']

In [10]:
postulantes_educacion_max = postulantes_educacion[idx][idx_postulantes_educacion_max]
len(postulantes_educacion_max)

366395

In [11]:
# Verificamos, por ejemplo para este postulante debe traer el maximo nombre y estado
postulantes_educacion[postulantes_educacion['idpostulante'] == '1QLLO'] 

,idpostulante,nombre_estudio,estado_estudio
5,1QLLO,4,2
290,1QLLO,4,3


In [12]:
# Efectivamente lo hace
postulantes_educacion_max[postulantes_educacion_max['idpostulante'] == '1QLLO'] 

,idpostulante,nombre_estudio,estado_estudio
290,1QLLO,4,3


In [13]:
postulantes_educacion_max = postulantes_educacion_max.apply(lambda row: descuantificar_estudios(row), axis=1)

In [14]:
postulantes_educacion_max.head(2)

,idpostulante,nombre_estudio,estado_estudio
1,1d2B,Universitario,En Curso
3,6M3jr,Universitario,En Curso


### Join con genero y edad

In [15]:
len(postulantes_genero_edad)

408146

In [16]:
len(postulantes_genero_edad.drop_duplicates(['idpostulante'], keep='last', inplace=False))

408146

In [17]:
postulantes = pd.merge(postulantes_educacion_max, postulantes_genero_edad, on='idpostulante', how='inner')

In [18]:
len(postulantes)

366395

In [19]:
postulantes.head(2)

,idpostulante,nombre_estudio,estado_estudio,fechanacimiento,sexo
0,1d2B,Universitario,En Curso,1976-02-28,MASC
1,6M3jr,Universitario,En Curso,1975-11-03,MASC


### Join con vistas y postulaciones

In [20]:
print(len(vistas))
print(len(postulaciones))

6682591
6604534


In [21]:
#vistas.rename(columns={'idAviso':'idaviso','timestamp':'fechavista'}, inplace=True)
vistas.head(1)

,idaviso,fechavista,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z


In [22]:
postulaciones.head(1)

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34


## POR AHORA NO MERGEO CON VISTAS, SOLO CON POSTULACIONES PORQUE ES DEMASIADO GRANDE

In [23]:
#vistas_postulaciones = pd.concat([vistas, postulaciones])
vistas_postulaciones = postulaciones

In [24]:
print(len(vistas_postulaciones))

6604534


In [25]:
vistas_postulaciones.to_csv('../data/vistas_postulaciones.csv', index=False, sep=',', encoding='utf-8')

In [26]:
postulantes.head(1)

,idpostulante,nombre_estudio,estado_estudio,fechanacimiento,sexo
0,1d2B,Universitario,En Curso,1976-02-28,MASC


In [27]:
vistas_postulaciones_postulantes = pd.merge(vistas_postulaciones, postulantes, on='idpostulante', how='outer')

In [28]:
print(len(vistas_postulaciones_postulantes))

6647748


In [29]:
vistas_postulaciones_postulantes.tail(2)

,idaviso,idpostulante,fechapostulacion,nombre_estudio,estado_estudio,fechanacimiento,sexo
6647746,NaN,DrXeJbA,NaN,Secundario,Abandonado,1993-01-20,MASC
6647747,NaN,VNr0VD2,NaN,Terciario/Técnico,Abandonado,1971-07-12,FEM


In [30]:
vistas_postulaciones_postulantes.to_csv('../data/postulantes_vistas_postulaciones.csv', index=False, sep=',', encoding='utf-8')

In [31]:
vistas_postulaciones_postulantes = pd.read_csv('../data/postulantes_vistas_postulaciones.csv')

In [32]:
vistas_postulaciones_postulantes.head(1)

,idaviso,idpostulante,fechapostulacion,nombre_estudio,estado_estudio,fechanacimiento,sexo
0,1.112257e+09,NM5M,2018-01-15 16:22:34,Secundario,Graduado,1970-12-03,FEM


In [33]:
len(vistas_postulaciones_postulantes)

6647748

### Merge con avisos_detalle

In [34]:
avisos_detalle.head(1)

,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,17903700,Enfermeras,"[('oeste', 2), ('experiencia', 2), ('años', 2)...",Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste


In [35]:
vistas_postulaciones_postulantes_avisos = pd.merge(vistas_postulaciones_postulantes, avisos_detalle, on='idaviso', how='left')

In [36]:
vistas_postulaciones_postulantes_avisos.to_csv('../data/vistas_postulaciones_postulantes_avisos.csv', index=False, sep=',', encoding='utf-8')

In [37]:
vistas_postulaciones_postulantes_avisos.head(1)

,idaviso,idpostulante,fechapostulacion,nombre_estudio,estado_estudio,fechanacimiento,sexo,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1.11226e+09,NM5M,2018-01-15 16:22:34,Secundario,Graduado,1970-12-03,FEM,REPRESENTANTES DE ATENCIÓN AL CLIENTE/ RETENCIÓN,"[('retencion', 3), ('atencion', 2), ('cliente'...",Part-time,Otro,Atención al Cliente,Eficasia Argentina SA


In [38]:
vistas_postulaciones_postulantes_avisos.columns

Index(['idaviso', 'idpostulante', 'fechapostulacion', 'nombre_estudio',
       'estado_estudio', 'fechanacimiento', 'sexo', 'titulo', 'descripcion',
       'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',
       'denominacion_empresa'],
      dtype='object')

In [39]:
vistas_postulaciones_postulantes_avisos.memory_usage(index=False)

idaviso                 54553152
idpostulante            54553152
fechapostulacion        54553152
nombre_estudio          54553152
estado_estudio          54553152
fechanacimiento         54553152
sexo                    54553152
titulo                  54553152
descripcion             54553152
tipo_de_trabajo         54553152
nivel_laboral           54553152
nombre_area             54553152
denominacion_empresa    54553152
dtype: int64